In [47]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium

In [48]:
fp = "C:/Users/Me/Desktop/brillicadata/geopandas/911calls/911/911.csv"

### Reading and parsing data

In [49]:
dateparse = lambda x: pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
df= pd.read_csv(fp,parse_dates=['timeStamp'],date_parser=dateparse)

In [50]:
df.head()


,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:10:52,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:29:21,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 14:39:21,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 16:47:36,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 16:56:52,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1


In [51]:

df.dtypes

lat                 float64
lng                 float64
desc                 object
zip                 float64
title                object
timeStamp    datetime64[ns]
twp                  object
addr                 object
e                     int64
dtype: object

### Resetting Index

In [52]:
df.set_index('timeStamp',inplace=True)
df.head(3)

,lat,lng,desc,zip,title,twp,addr,e
timeStamp,,,,,,,,
2015-12-10 17:10:52,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,NEW HANOVER,REINDEER CT & DEAD END,1
2015-12-10 17:29:21,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2015-12-10 14:39:21,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,NORRISTOWN,HAWS AVE,1


In [53]:
df.dtypes

lat      float64
lng      float64
desc      object
zip      float64
title     object
twp       object
addr      object
e          int64
dtype: object

### Defining 'type' attribute by spliting title

In [54]:
df['type'] = df["title"].apply(lambda x: x.split(':')[0].strip())

In [55]:
df.head(3)

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 17:10:52,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,NEW HANOVER,REINDEER CT & DEAD END,1,EMS
2015-12-10 17:29:21,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,EMS
2015-12-10 14:39:21,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,NORRISTOWN,HAWS AVE,1,Fire


In [56]:
print("The frequency of emergency type is:\n",df["type"].value_counts())

The frequency of emergency type is:
 EMS        208676
Traffic    151458
Fire        63775
Name: type, dtype: int64


# Dividing Data Based on It's Type EMS, Traffic, Fire.

In [57]:
# Extracting data for EMS
ems_city = df[df['type']=='EMS']

In [58]:
ems_city.shape

(208676, 9)

In [59]:
ems_city.head()

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 17:10:52,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,NEW HANOVER,REINDEER CT & DEAD END,1,EMS
2015-12-10 17:29:21,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,EMS
2015-12-10 16:47:36,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,NORRISTOWN,AIRY ST & SWEDE ST,1,EMS
2015-12-10 16:56:52,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1,EMS
2015-12-10 15:39:04,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,LANSDALE,CANNON AVE & W 9TH ST,1,EMS


In [60]:
# Extracting data for Traffic
traffic_city =df[df['type']=='Traffic']

In [61]:
traffic_city.shape

(151458, 9)

In [62]:
traffic_city.head()

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 17:35:41,40.102398,-75.291458,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD; PLYM...,19462.0,Traffic: VEHICLE ACCIDENT -,PLYMOUTH,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD,1,Traffic
2015-12-10 17:33:50,40.231990,-75.251891,RT202 PKWY & KNAPP RD; MONTGOMERY; 2015-12-10 ...,NaN,Traffic: VEHICLE ACCIDENT -,MONTGOMERY,RT202 PKWY & KNAPP RD,1,Traffic
2015-12-10 16:32:10,40.084161,-75.308386,BROOK RD & COLWELL LN; PLYMOUTH; 2015-12-10 @ ...,19428.0,Traffic: VEHICLE ACCIDENT -,PLYMOUTH,BROOK RD & COLWELL LN,1,Traffic
2015-12-10 17:15:49,40.174131,-75.098491,BYBERRY AVE & S WARMINSTER RD; UPPER MORELAND;...,19040.0,Traffic: VEHICLE ACCIDENT -,UPPER MORELAND,BYBERRY AVE & S WARMINSTER RD,1,Traffic
2015-12-10 17:12:47,40.062974,-75.135914,OLD YORK RD & VALLEY RD; CHELTENHAM; 2015-12-1...,19027.0,Traffic: VEHICLE ACCIDENT -,CHELTENHAM,OLD YORK RD & VALLEY RD,1,Traffic


In [63]:
# Extracting Data for Fire
fire_city = df[df['type']=='Fire']

In [64]:
fire_city.shape

(63775, 9)

In [65]:
fire_city.head()

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 14:39:21,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,NORRISTOWN,HAWS AVE,1,Fire
2015-12-10 17:59:24,40.199006,-75.300058,LILAC CT & PRIMROSE DR; UPPER GWYNEDD; 2015-12...,19446.0,Fire: APPLIANCE FIRE,UPPER GWYNEDD,LILAC CT & PRIMROSE DR,1,Fire
2015-12-10 18:05:19,40.028903,-75.351822,EDENTON PL & DURHAM DR; DELAWARE COUNTY; 2015-...,19085.0,Fire: CARBON MONOXIDE DETECTOR,DELAWARE COUNTY,EDENTON PL & DURHAM DR,1,Fire
2015-12-10 18:32:25,40.104206,-75.367665,HAMPTON RD & BELMONT RD; UPPER MERION; 2015-12...,19406.0,Fire: GAS-ODOR/LEAK,UPPER MERION,HAMPTON RD & BELMONT RD,1,Fire
2015-12-10 18:50:23,40.224923,-75.528045,LINFIELD TRAPPE RD; LIMERICK; 2015-12-10 @ 18:...,19468.0,Fire: VEHICLE ACCIDENT,LIMERICK,LINFIELD TRAPPE RD,1,Fire


# Traffic Call Locations on Map

In [66]:
gdf_traffic = gpd.GeoDataFrame(
    traffic_city)
gdf_traffic.head()

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 17:35:41,40.102398,-75.291458,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD; PLYM...,19462.0,Traffic: VEHICLE ACCIDENT -,PLYMOUTH,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD,1,Traffic
2015-12-10 17:33:50,40.231990,-75.251891,RT202 PKWY & KNAPP RD; MONTGOMERY; 2015-12-10 ...,NaN,Traffic: VEHICLE ACCIDENT -,MONTGOMERY,RT202 PKWY & KNAPP RD,1,Traffic
2015-12-10 16:32:10,40.084161,-75.308386,BROOK RD & COLWELL LN; PLYMOUTH; 2015-12-10 @ ...,19428.0,Traffic: VEHICLE ACCIDENT -,PLYMOUTH,BROOK RD & COLWELL LN,1,Traffic
2015-12-10 17:15:49,40.174131,-75.098491,BYBERRY AVE & S WARMINSTER RD; UPPER MORELAND;...,19040.0,Traffic: VEHICLE ACCIDENT -,UPPER MORELAND,BYBERRY AVE & S WARMINSTER RD,1,Traffic
2015-12-10 17:12:47,40.062974,-75.135914,OLD YORK RD & VALLEY RD; CHELTENHAM; 2015-12-1...,19027.0,Traffic: VEHICLE ACCIDENT -,CHELTENHAM,OLD YORK RD & VALLEY RD,1,Traffic


In [67]:
gdf_traffic.dtypes

lat      float64
lng      float64
desc      object
zip      float64
title     object
twp       object
addr      object
e          int64
type      object
dtype: object

In [68]:
new_geo = gpd.GeoDataFrame()
new_geo['lat']=gdf_traffic['lat'].astype('float64')
new_geo['lng']=gdf_traffic['lng'].astype('float64')
new_geo['title']=gdf_traffic['title'].astype('object')
location = new_geo['lat'].mean(), new_geo['lng'].mean()

new_geo.shape

(151458, 3)

In [69]:
new_geo.head()


,lat,lng,title
timeStamp,,,
2015-12-10 17:35:41,40.102398,-75.291458,Traffic: VEHICLE ACCIDENT -
2015-12-10 17:33:50,40.231990,-75.251891,Traffic: VEHICLE ACCIDENT -
2015-12-10 16:32:10,40.084161,-75.308386,Traffic: VEHICLE ACCIDENT -
2015-12-10 17:15:49,40.174131,-75.098491,Traffic: VEHICLE ACCIDENT -
2015-12-10 17:12:47,40.062974,-75.135914,Traffic: VEHICLE ACCIDENT -


In [70]:
locationlist = new_geo[['lat','lng']].values.tolist()
labels = "CITY => "+ gdf_traffic['twp']+"\n"+gdf_traffic['title']

m = folium.Map(location=location, zoom_start=10)


for point in range(1,100): 
    popup = folium.Popup(labels[point], parse_html=True)
    icon = folium.Icon(color='orange')
    folium.Marker(locationlist[point], popup=popup, icon=icon).add_to(m)
    
m.save(outfile= "Traffic.html")
m



# Fire Call Locations on Map

In [71]:
gdf_fire = gpd.GeoDataFrame(
    fire_city)
gdf_fire.head()

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 14:39:21,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,NORRISTOWN,HAWS AVE,1,Fire
2015-12-10 17:59:24,40.199006,-75.300058,LILAC CT & PRIMROSE DR; UPPER GWYNEDD; 2015-12...,19446.0,Fire: APPLIANCE FIRE,UPPER GWYNEDD,LILAC CT & PRIMROSE DR,1,Fire
2015-12-10 18:05:19,40.028903,-75.351822,EDENTON PL & DURHAM DR; DELAWARE COUNTY; 2015-...,19085.0,Fire: CARBON MONOXIDE DETECTOR,DELAWARE COUNTY,EDENTON PL & DURHAM DR,1,Fire
2015-12-10 18:32:25,40.104206,-75.367665,HAMPTON RD & BELMONT RD; UPPER MERION; 2015-12...,19406.0,Fire: GAS-ODOR/LEAK,UPPER MERION,HAMPTON RD & BELMONT RD,1,Fire
2015-12-10 18:50:23,40.224923,-75.528045,LINFIELD TRAPPE RD; LIMERICK; 2015-12-10 @ 18:...,19468.0,Fire: VEHICLE ACCIDENT,LIMERICK,LINFIELD TRAPPE RD,1,Fire


In [72]:
new_geo = gpd.GeoDataFrame()
new_geo['lat']=gdf_fire['lat'].astype('float64')
new_geo['lng']=gdf_fire['lng'].astype('float64')
location = new_geo['lat'].mean(), new_geo['lng'].mean()

new_geo.shape

(63775, 2)

In [73]:
new_geo.head()

,lat,lng
timeStamp,,
2015-12-10 14:39:21,40.121182,-75.351975
2015-12-10 17:59:24,40.199006,-75.300058
2015-12-10 18:05:19,40.028903,-75.351822
2015-12-10 18:32:25,40.104206,-75.367665
2015-12-10 18:50:23,40.224923,-75.528045


In [74]:
locationlist = new_geo[['lat','lng']].values.tolist()

labels = "CITY => "+ gdf_fire['twp']+"\n"+gdf_fire['title']

m = folium.Map(location=location, zoom_start=10)

for point in range(1,100): 
    popup = folium.Popup(labels[point], parse_html=True)
    icon = folium.Icon(color='purple')
    folium.Marker(locationlist[point], popup=popup,icon=icon).add_to(m)

m.save(outfile="Fire.html")
m

# EMS Call Locations on Map

In [75]:
gdf_ems = gpd.GeoDataFrame(
    ems_city)
gdf_ems.head()

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 17:10:52,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,NEW HANOVER,REINDEER CT & DEAD END,1,EMS
2015-12-10 17:29:21,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,EMS
2015-12-10 16:47:36,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,NORRISTOWN,AIRY ST & SWEDE ST,1,EMS
2015-12-10 16:56:52,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1,EMS
2015-12-10 15:39:04,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,LANSDALE,CANNON AVE & W 9TH ST,1,EMS


In [76]:
new_geo = gpd.GeoDataFrame()
new_geo['lat']=gdf_ems['lat'].astype('float64')
new_geo['lng']=gdf_ems['lng'].astype('float64')
location = new_geo['lat'].mean(), new_geo['lng'].mean()

new_geo.shape

(208676, 2)

In [77]:
new_geo.head()

,lat,lng
timeStamp,,
2015-12-10 17:10:52,40.297876,-75.581294
2015-12-10 17:29:21,40.258061,-75.264680
2015-12-10 16:47:36,40.116153,-75.343513
2015-12-10 16:56:52,40.251492,-75.603350
2015-12-10 15:39:04,40.253473,-75.283245


In [78]:
locationlist = new_geo[['lat','lng']].values.tolist()

labels = "CITY => "+ gdf_ems['twp']+"\n"+gdf_ems['title']

m = folium.Map(location=location, zoom_start=10)

for point in range(1,100): 
    popup = folium.Popup(labels[point], parse_html=True)
    icon = folium.Icon(color='blue')
    folium.Marker(locationlist[point], popup=popup,icon=icon).add_to(m)
    
m.save(outfile= "EMS.html")
m

# All Maps in One

In [79]:
gdf = gpd.GeoDataFrame(
    df)
gdf.head()

,lat,lng,desc,zip,title,twp,addr,e,type
timeStamp,,,,,,,,,
2015-12-10 17:10:52,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,NEW HANOVER,REINDEER CT & DEAD END,1,EMS
2015-12-10 17:29:21,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1,EMS
2015-12-10 14:39:21,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,NORRISTOWN,HAWS AVE,1,Fire
2015-12-10 16:47:36,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,NORRISTOWN,AIRY ST & SWEDE ST,1,EMS
2015-12-10 16:56:52,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1,EMS


In [80]:
new_geo = gpd.GeoDataFrame()
new_geo['lat']=gdf['lat'].astype('float64')
new_geo['lng']=gdf['lng'].astype('float64')
new_geo['type']=gdf['type'].astype('object')
location = new_geo['lat'].mean(), new_geo['lng'].mean()

new_geo.shape

(423909, 3)

In [81]:
locationlist = new_geo[['lat','lng']].values.tolist()
labels = "CITY => "+ gdf['twp']+"\n"+gdf['title']
etype = gdf['type'].values.tolist()

m = folium.Map(location=location, zoom_start=10)

for point in range(1,300):
    if(etype[point] == 'EMS'):
        icon = folium.Icon(color='blue')
        label=labels[point]
    elif(etype[point]=='Traffic'):
        icon = folium.Icon(color='purple')
        label=labels[point]
    elif(etype[point]=='Fire'):
        icon = folium.Icon(color='red')
        label=labels[point]
    
    popup = folium.Popup(label, parse_html=True)
    folium.Marker(locationlist[point], popup=popup, icon=icon).add_to(m)
    
m.save(outfile= "All-Map-In-One.html")
m